In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
import tritonclient
import tritonclient.http as httpclient
from tritonclient.utils import triton_to_np_dtype

## Check the Model Status

In [ ]:
model_name = 'endo_seg'
!curl http://10.19.104.11:7000/v2/models/$MODEL_NAME/versions/1/stats

## Execute Inference Service

In [ ]:
import monai

client = httpclient
client_inferer = client.InferenceServerClient(url='10.19.104.11:7000', verbose=0)

def infer(data, model_name, input_name, output_name, priority=0):
    inputs = [client.InferInput(input_name, data.shape, 'FP32')]
    inputs[0].set_data_from_numpy(data)
    outputs = [client.InferRequestedOutput(output_name, class_count=0)]
    result = client_inferer.infer(
            model_name=model_name,
            inputs=inputs, 
            request_id=str(1),
            model_version='1',
            outputs=outputs,
            priority=priority)
    return result

- Load image

In [ ]:
data = {'img':'./sample.jpg'}
spatial_size = (256, 256)
trans = monai.transforms.Compose([monai.transforms.LoadImaged('img'), 
                                  monai.transforms.EnsureChannelFirstd('img'),
                                  monai.transforms.Resized('img', spatial_size=spatial_size, mode='area'),
                                  monai.transforms.NormalizeIntensityd('img'),
                                  monai.transforms.EnsureTyped('img', data_type='numpy')])
post_trans = monai.transforms.AsDiscrete(threshold=0.5)
data = trans(data)
data['img'].shape

In [ ]:
# Batching
batch_size = 32
inputs = np.array([data['img'] for i in range(batch_size)])
inputs.shape

- Setup

In [ ]:
input_name = 'input.1'
output_name = '495'

- Inference

In [ ]:
result = infer(inputs, model_name, input_name, output_name)
result = post_trans(result.as_numpy(output_name))
result.shape

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(inputs[0].transpose([1, 2, 0]))
plt.subplot(122)
plt.imshow(result[0, 0], cmap='gray')

## Exercise
Simulate clinical usage! The link below contains some endoscopy videos. Please do inference on the videos frame by frame, and show the segmentation result runtime.
- Link: http://www.depeca.uah.es/colonoscopy_dataset/